In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_excel("/content/data1.xlsx", sheet_name = 'Data')

In [3]:
df.head()

,index,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,...,x_14,x_15,x_16,x_17,x_18,x_19,x_20,x_21,x_22,target
0,0,10.528055,0.308325,1154.451034,0.147287,0.854394,4.483094,9.894689,28.741196,389.133299,...,12.143896,10.328332,1.756277,32.440867,4.767277,25.931299,67.881356,7.790180e+06,7.297312e+02,0
1,1,8.932895,0.000000,1321.542631,0.189447,0.854394,4.483094,11.543803,11.676111,391.040816,...,14.572675,10.328332,8.781386,571672.962983,14.301832,185.223564,3256.042367,8.722124e+06,2.949072e+07,1
2,2,6.460397,0.308325,1184.831324,0.136356,3.417574,4.483094,7.146164,15.268760,495.000447,...,12.143896,5.901904,1.756277,0.000000,38.138219,1196.544226,730.855931,6.396224e+05,0.000000e+00,0
3,3,14.675470,1.849952,0.000000,0.127774,5.980755,10.759425,8.795279,31.435683,980.463313,...,19.430234,1.475476,1.756277,16.220434,0.000000,25.931299,15.838983,4.024388e+07,1.060543e+03,0
4,4,6.938945,1.849952,0.000000,0.114831,0.854394,5.977458,7.695869,110.473971,768.729018,...,4.857558,0.000000,1.756277,16.220434,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0


In [4]:
# общая оценка датасета даёт понимание по выбросам
df.describe()

,index,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,...,x_14,x_15,x_16,x_17,x_18,x_19,x_20,x_21,x_22,target
count,37163.000000,37163.000000,37163.000000,37163.000000,37163.000000,37163.000000,29659.000000,37163.000000,37163.000000,37163.000000,...,36771.000000,36771.000000,36762.000000,3.676200e+04,36762.000000,36762.000000,36762.000000,3.563700e+04,3.563700e+04,37163.000000
mean,18684.912494,9.504722,1.547044,12.789184,0.160645,2.127006,9.809432,10.679249,64.973031,840.680509,...,12.044654,13.162197,1.804290,4.391995e+03,5.978549,541.152294,760.318517,6.796170e+06,3.758539e+05,0.018029
std,10782.815759,4.548465,0.476993,48.567391,0.030803,1.817300,7.329434,2.935218,58.356331,801.805834,...,14.626903,19.420506,1.584417,7.124006e+04,14.250700,2004.683136,1720.381726,1.975901e+07,3.656964e+06,0.133057
min,0.000000,0.000000,0.000000,0.000000,0.042327,0.854394,0.747182,5.222197,4.490812,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
25%,9350.500000,6.061607,1.541627,0.000000,0.140838,0.854394,5.230276,8.245574,13.472435,310.925136,...,3.643169,2.950952,0.878139,0.000000e+00,0.000000,0.000000,2.262712,0.000000e+00,0.000000e+00,0.000000
50%,18684.000000,10.528055,1.849952,0.000000,0.169732,0.854394,8.219005,10.169541,41.315469,443.497510,...,7.286338,8.852856,1.756277,1.622043e+01,0.000000,7.408943,42.991525,1.260498e+06,1.654057e+02,0.000000
75%,28019.500000,13.877891,1.849952,7.595073,0.186491,3.417574,12.453038,12.918066,110.473971,1081.561670,...,14.572675,16.230236,2.634416,1.622043e+01,4.767277,85.202840,500.059321,6.285744e+06,5.837850e+02,0.000000
max,37361.000000,14.675470,3.699904,1830.412494,0.503493,7.689542,112.824525,18.140263,359.264946,3241.823737,...,268.380106,674.292553,34.247406,2.286692e+06,395.684025,22375.006588,84369.737133,1.540909e+09,1.327082e+08,1.000000


In [5]:
# оценка отсутствующих значений
df.isnull().sum()

index         0
x_0           0
x_1           0
x_2           0
x_3           0
x_4           0
x_5        7504
x_6           0
x_7           0
x_8           0
x_9           0
x_10       5363
x_11      22728
x_12         32
x_13        392
x_14        392
x_15        392
x_16        401
x_17        401
x_18        401
x_19        401
x_20        401
x_21       1526
x_22       1526
target        0
dtype: int64

In [6]:
df = df.drop(['x_11'], axis=1)

In [7]:
df = df.fillna(0)
df.isnull().sum()

index     0
x_0       0
x_1       0
x_2       0
x_3       0
x_4       0
x_5       0
x_6       0
x_7       0
x_8       0
x_9       0
x_10      0
x_12      0
x_13      0
x_14      0
x_15      0
x_16      0
x_17      0
x_18      0
x_19      0
x_20      0
x_21      0
x_22      0
target    0
dtype: int64

In [8]:
df.nunique()

index     37163
x_0         185
x_1           8
x_2         174
x_3       37053
x_4           9
x_5         496
x_6          48
x_7         288
x_8        2823
x_9         942
x_10      20553
x_12      35888
x_13        614
x_14        131
x_15        171
x_16         30
x_17         19
x_18         92
x_19       2101
x_20       2987
x_21      12765
x_22        629
target        2
dtype: int64

In [9]:
# оценка распределения значений целевой переменной
df.target.value_counts(normalize=True)

0    0.981971
1    0.018029
Name: target, dtype: float64

In [10]:
# Разбиваем на тестовую и обучающую выборки
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=11)
X_train, X_val = train_test_split(df_train_full, test_size=0.25, random_state=11)

In [11]:
y_train = X_train['target'].values
y_val = X_val['target'].values

In [12]:
del X_train['target']
del X_val['target']

In [25]:
# Модель логистическая регрессия
lr = LogisticRegression(solver='liblinear', random_state=1)
lr.fit(X_train, y_train)

y_pred_lr = lr.predict_proba(X_train)[:, 1]
auc = roc_auc_score(y_train, y_pred_lr)
print('train auc: %.3f' % auc)

y_pred_lr = lr.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, y_pred_lr)
print('val auc: %.3f' % auc)

train auc: 0.610
val auc: 0.621


In [27]:
# Модель дерево решений
dt = DecisionTreeClassifier(max_depth=5, min_samples_leaf = 200)
dt.fit(X_train, y_train)

y_pred_dt = dt.predict_proba(X_train)[:, 1]
auc = roc_auc_score(y_train, y_pred_dt)
print('train auc: %.3f' % auc)

y_pred_dt = dt.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, y_pred_dt)
print('val auc: %.3f' % auc)

train auc: 0.863
val auc: 0.836


In [15]:
# Подбор параметров, наилучшая оценка модели
params = {
    "max_depth": [1, 2, 3, 4, 5, 6, 10, 15, 20],
    "min_samples_leaf": [1, 5, 10, 15, 20, 50, 100, 200]
}
gs = GridSearchCV(dt, params, scoring='roc_auc')
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(max_depth=2),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 10, 15, 20],
                         'min_samples_leaf': [1, 5, 10, 15, 20, 50, 100, 200]},
             scoring='roc_auc')

In [16]:
gs.best_params_

{'max_depth': 5, 'min_samples_leaf': 200}

In [17]:
gs.best_score_

0.8185523135901676

In [28]:
# Модель случайный лес
rf = RandomForestClassifier(max_depth = 5, min_samples_leaf = 5, n_estimators=180, random_state=3)
rf.fit(X_train, y_train)

y_pred_rf = dt.predict_proba(X_train)[:, 1]
auc = roc_auc_score(y_train, y_pred_rf)
print('train auc: %.3f' % auc)

y_pred_rf = dt.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, y_pred_rf)
print('val auc: %.3f' % auc)

train auc: 0.863
val auc: 0.836


In [19]:
params = {
    "max_depth": [5, 10, 20],
    "n_estimators": [10, 30, 50, 100, 130, 150, 180, 200],
    "min_samples_leaf": [3, 5, 10]
}
gs = GridSearchCV(rf, params, scoring='roc_auc')
gs.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(n_estimators=10, random_state=3),
             param_grid={'max_depth': [5, 10, 20],
                         'min_samples_leaf': [3, 5, 10],
                         'n_estimators': [10, 30, 50, 100, 130, 150, 180, 200]},
             scoring='roc_auc')

In [20]:
gs.best_params_

{'max_depth': 5, 'min_samples_leaf': 5, 'n_estimators': 180}

In [21]:
gs.best_score_

0.854407815522243

In [22]:
# Модель градиентного бустинга XGBoost 
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

xgb_params = {
    'eta': 0.1,
    'max_depth': 3,
    'min_child_weight': 1,

    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'nthread': 8,
    'seed': 1,
}

model = xgb.train(xgb_params, dtrain,
                  num_boost_round=180)

In [23]:
y_pred_xgb = model.predict(dtrain)
auc = roc_auc_score(y_train, y_pred_xgb)
print('train auc: %.3f' % auc)

y_pred_xgb = model.predict(dval)
auc = roc_auc_score(y_val, y_pred_xgb)
print('val auc: %.3f' % auc)

train auc: 0.948
val auc: 0.870


In [26]:
# сравнение результатов моделей до подбора параметров

print(roc_auc_score(y_val, y_pred_lr))
print(roc_auc_score(y_val, y_pred_dt))
print(roc_auc_score(y_val, y_pred_rf))
print(roc_auc_score(y_val, y_pred_xgb))

0.6205644580867792
0.7949136914835262
0.7949136914835262
0.870349027205565


In [29]:
# сравнение результатов моделей после подбора параметров

print(roc_auc_score(y_val, y_pred_lr))
print(roc_auc_score(y_val, y_pred_dt))
print(roc_auc_score(y_val, y_pred_rf))
print(roc_auc_score(y_val, y_pred_xgb))

0.6205644580867792
0.8362968452749151
0.8362968452749151
0.870349027205565


In [30]:
# финальный тест
y_train_full = df_train_full['target'].values
y_test = df_test['target'].values

del df_train_full['target']
del df_test['target']

In [31]:
# финальная модель
dtrain_full = xgb.DMatrix(df_train_full, label=y_train_full)
dtest = xgb.DMatrix(df_test, label=y_test)

xgb_params = {
    'eta': 0.1,
    'max_depth': 3,
    'min_child_weight': 1,

    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'nthread': 8,
    'seed': 1,
}

xgb_final = xgb.train(xgb_params, dtrain_full, num_boost_round=160)

In [32]:
# финальная оценка
y_pred_final = model.predict(dtrain_full)
auc = roc_auc_score(y_train_full, y_pred_final)
print('train auc: %.3f' % auc)

y_pred_final = model.predict(dtest)
auc = roc_auc_score(y_test, y_pred_final)
print('val auc: %.3f' % auc)

train auc: 0.928
val auc: 0.871
